# 📦 Kütüphanelerin İçe Aktarılması

In [1]:
# 📦 Kütüphanelerin İçe Aktarılması

# TensorFlow kütüphanesini içe aktarıyoruz.
# Derin öğrenme modellerini oluşturmak, eğitmek ve değerlendirmek için kullanılır.
import tensorflow as tf

# Fashion MNIST veri setini içeren Keras modülünü içe aktarıyoruz.
# Bu veri seti, giyim eşyalarına ait 28x28 boyutunda gri tonlamalı görüntüler içerir.
from tensorflow.keras.datasets import fashion_mnist

# Modelde kullanacağımız katmanları içe aktarıyoruz:
# Conv2D - evrişimsel katman, BatchNormalization - normalizasyon katmanı,
# Activation - aktivasyon fonksiyonu, Flatten - veriyi düzleştirme,
# Dense - tam bağlantılı katman, Input - giriş tanımlayıcı, Add - katmanları toplama.
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Flatten, Dense, Input, Add

# Model sınıfını içe aktarıyoruz. Keras fonksiyonel API ile özel modeller inşa etmek için kullanılır.
from tensorflow.keras.models import Model

# Adam optimizasyon algoritmasını içe aktarıyoruz.
# Derin öğrenme modellerinde sıkça kullanılan ve genellikle iyi sonuç veren bir optimizer’dır.
from tensorflow.keras.optimizers import Adam

# Sınıf etiketlerini one-hot encoding formatına dönüştürmek için kullanılır.
from tensorflow.keras.utils import to_categorical

# Keras Tuner'dan HyperModel ve RandomSearch sınıflarını içe aktarıyoruz.
# Hiperparametre araması yapmak için kullanılır. HyperModel, modelin yapısını tanımlar,
# RandomSearch ise çeşitli hiperparametre kombinasyonlarını deneyerek en iyisini bulur.
from keras_tuner import HyperModel, RandomSearch

# 📊 Verinin Yüklenmesi ve Ön İşlenmesi

In [2]:
# Fashion MNIST veri setini yükleyip eğitim ve test verilerine ayırıyoruz.
# train_images ve train_labels: Eğitim için kullanılacak görüntüler ve etiketler.
# test_images ve test_labels: Modelin test edilmesi için kullanılacak veriler.
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Eğitim verilerindeki görüntüleri 4 boyutlu hale getiriyoruz (örnek sayısı, yükseklik, genişlik, kanal).
# Ardından float32 formatına çevirip 255'e bölerek 0-1 arasında normalize ediyoruz.
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Test verileri için de aynı yeniden şekillendirme ve normalize işlemi uygulanıyor.
test_images = test_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Eğitim etiketlerini one-hot encoding formatına çeviriyoruz.
# Örneğin; etiket "3" ise [0, 0, 0, 1, 0, 0, 0, 0, 0, 0] haline gelir.
train_labels = to_categorical(train_labels, 10)

# Aynı şekilde test etiketleri de one-hot encoding'e dönüştürülüyor.
test_labels = to_categorical(test_labels, 10)

# 🧱 Artık (Residual) Blok Fonksiyonu Tanımı

In [3]:
# Bu fonksiyon, derin öğrenme modellerinde kullanılan bir residual (artık) blok yapısını tanımlar.
# Amaç: Girdi ile öğrenilen çıktıyı toplamak (skip connection) ve böylece daha derin ağlarda
# gradyanların kaybolma sorununu azaltmaktır.
def residual_block(x, filters, kernel_size=3, stride=1):
    
    # shortcut değişkeni, girdiyi daha sonra eklemek için saklar.
    shortcut = x

    # İlk Conv2D katmanı: Görüntü filtresi uygulanır.
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)  # Normalizasyon uygulanır.
    x = Activation("relu")(x)    # Aktivasyon fonksiyonu olarak ReLU kullanılır.

    # İkinci Conv2D katmanı: Aynı işlemler tekrarlanır.
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Eğer giriş ve çıkış katmanlarının kanal (filter) sayısı farklıysa, shortcut da dönüştürülmelidir.
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding="same")(shortcut)
        shortcut = BatchNormalization()(shortcut)
        shortcut = Activation("relu")(shortcut)

    # Girdi (shortcut) ve işlenmiş çıktı toplanır.
    x = Add()([x, shortcut])

    # Son bir ReLU aktivasyonu uygulanır.
    x = Activation("relu")(x)

    # Çıktı olarak residual blok döndürülür.
    return x


# 🧠ResNetModel Sınıfı (HyperModel ile Hyperparameter Tuning için)

In [4]:
# Bu sınıf, Keras Tuner ile birlikte çalışmak üzere tanımlanmış özel bir model yapısıdır.
# Amaç: Farklı hiperparametre kombinasyonlarıyla ResNet benzeri bir model oluşturmak.

class ResNetModel(HyperModel):

    # build metodu: Model mimarisi burada tanımlanır. Hiperparametreler burada alınır.
    def build(self, hp):
        
        # Giriş katmanı: Girdi şekli Fashion MNIST datasına uygun (28x28x1)
        inputs = Input(shape=[28, 28, 1])

        # İlk konvolüsyonel katman: Hiperparametre olarak filtre sayısı belirlenir
        x = Conv2D(
            filters=hp.Int("initial_filters", min_value=32, max_value=128, step=32),
            kernel_size=3,
            padding="same",
            activation="relu"
        )(inputs)
        x = BatchNormalization()(x)  # Normalizasyon uygulanır

        # Belirli sayıda residual block eklenir (1 ila 3 arasında)
        for i in range(hp.Int("num_blocks", min_value=1, max_value=3, step=1)):
            # Her block için farklı filtre sayısı seçilir
            x = residual_block(
                x,
                hp.Int("res_filters" + str(i), min_value=32, max_value=128, step=32)
            )

        # Özellikler düzleştirilir ve yoğun katmanlara geçilir
        x = Flatten()(x)
        x = Dense(128, activation="relu")(x)  # Gizli katman
        outputs = Dense(10, activation="softmax")(x)  # Çıkış katmanı (10 sınıf için)

        # Model oluşturulur
        model = Model(inputs, outputs)

        # Model derlenir: Öğrenme oranı hiperparametre olarak verilir
        model.compile(
            optimizer=Adam(hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="LOG")),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        return model

# 🚀 Model Taraması (Hyperparameter Tuning)

In [5]:
# Bu aşamada Keras Tuner kullanılarak modelin en iyi hiperparametre kombinasyonu aranır.

tuner = RandomSearch(
    ResNetModel(),                          # Kullanılacak model sınıfı (HyperModel)
    objective="val_accuracy",              # Hedeflenen metrik: doğruluk (validation datası üzerinde)
    max_trials=2,                          # Denenecek farklı hiperparametre kombinasyonu sayısı
    executions_per_trial=1,               # Her deneme için kaç kez eğitileceği
    directory="Resnet_Hyperparameter_Tuning_Directory",  # Kayıt dizini
    project_name="resnet_model_tuning"     # Proje adı (alt klasör olarak oluşturulur)
)

# Model taraması başlatılır
tuner.search(
    train_images,                          # Eğitim görüntüleri
    train_labels,                          # Eğitim etiketleri
    epochs=1,                              # Her model için epoch sayısı
    validation_data=(test_images, test_labels)  # Doğrulama verisi
)

Reloading Tuner from Resnet_Hyperparameter_Tuning_Directory\resnet_model_tuning\tuner0.json


# 🏆 En İyi Modelin Seçilmesi


In [6]:
best_model = tuner.get_best_models(num_models=1)[0]

C:\Users\Teymur Mammadov\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# 📊Modelin Test Verisi Üzerinde Değerlendirilmesi

In [7]:
# Eğitilen ve hiperparametreleri optimize edilen modelin, daha önce hiç görmediği test verisi üzerindeki performansı ölçülür.

test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)
# evaluate() fonksiyonu, test verilerini kullanarak modelin kaybını (loss) ve doğruluk oranını (accuracy) döner.

print(f"Test loss: {test_loss:.2f}, Test accuracy: {test_accuracy:.2f}")
# Test sonuçları iki ondalık basamakla yazdırılır.

313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 71ms/step - accuracy: 0.8839 - loss: 0.3287
Test loss: 0.33, Test accuracy: 0.88
